# 🫠 C#'s Multiple Personality Disorder

## Properties

In [ ]:
class Node<T>
{
    T variable;

    public T GetValue()
    {
        return this.variable;
    }

    public Node(T variable)
    {
        this.variable = variable;
    }
}

In [ ]:
class Node<T>
{
    T variable;

    public T Value
    {
        get
        {
            return this.variable;
        }
    }

    public Node(T variable)
    {
        this.variable = variable;
    }
}

In [ ]:
class Node<T>
{
    public required T Value { get; init; } // we can have private set;

    public Node(T variable)
    {
        this.Value = variable;
    }
}

In [ ]:
class Node<T>
{
    T variable;

    public T Value => this.variable;
    public Node(T variable) => (this.variable) = variable;
}

## Delegates

In [ ]:
// C# 1: Traditional Delegate Declaration and Usage
delegate int MathOperation(int x, int y);
int Add1(int x, int y) => x + y; // I am using newer syntax
MathOperation operation1 = new MathOperation(Add1);
int result1 = operation1(3, 4); // operation.Invoke(3, 4); operation?.Invoke

// C# 2: Anonymous Methods
MathOperation operation2 = delegate (int x, int y) { return x + y; };

// C# 3: Lambdra Expressions
MathOperation operation3 = (x, y) => { return x + y; }; // in modern syntax we can do (x, y) => x + y
// Also had Action, Func; no need to define delegate "types"
Func<int, int, int> operation3b = (x, y) => x + y; // i am using newer syntax

// C# 6: Expression-Bodied Members
int Add6(int x, int y) => x + y;
MathOperation operation6a = Add6;
MathOperation operation6b = (x, y) => x + y;

// C# 9: Target-Typed New (Eliminating Redundancy)
MathOperation operation9 = new((x, y) => x + y);

In [ ]:
var operation = (int x, int y) => x + y; // inference

## ⚡Succinctness

- F# Effect

# 🤔 Maybe Pattern

In [ ]:
// Haskell
data Maybe a = Nothing | Just a

// Multi Bodied Functions
safeDivide :: Double -> Double -> Maybe Double
safeDivide _ 0 = Nothing
safeDivide x y = Just (x / y)

- __Verbosity__: Handling Maybe values can become verbose compared to implicit null handling, especially in large codebases without concise helper functions
- __Overuse__: Using Maybe where absence is logically impossible can lead to unnecessary complexity
- __Error Propagation__: While Maybe is excellent for propagating errors, debugging the actual cause of Nothing requires extra effort

In [ ]:
// Scala
val maybeValue: Option[Int] = Some(42)
val result = maybeValue.map(_ * 2)

// Rust
let maybe_value: Option<i32> = Some(42);
let result = maybe_value.map(|x| x * 2);

// Java
Optional<Integer> maybeValue = Optional.of(42);
Optional<Integer> result = maybeValue.map(x -> x * 2);

__Why Learn "Maybe" Even if You Don't Use Haskell__

- __Conceptual Clarity__: Understanding Maybe enhances your grasp of functional programming concepts like monads, type safety, and composability
- __Better Error Handling__: You can apply similar constructs in your preferred languages (like Nullable<T> in C#)
- __Modern Language Trends__: Many languages are moving toward safer handling of nullability. Learning Maybe prepares you for this paradigm shift
- __Improved Code Design__: The principles behind Maybe encourage explicitness and immutability, improving overall code quality

Understanding Maybe equips you with tools to handle one of the most common sources of bugs — absence of values — in a principled, type-safe way. Even if you don't write Haskell daily, the ideas translate well into other languages, enriching your overall programming skill set

## C#

- Java and C# have Nullable and Optional types

In [ ]:
Nullable<int> v1 = 5;
int? v2 = null; // shortened syntax

if (v1.HasValue) Console.WriteLine(v1.Value);
if (v2.HasValue) Console.WriteLine(v2.Value);

// Java
//Integer i = null; // similar to C# you can get NullPointerException at runtime if you are not explicitly checking for nulls
//Optional<Integer> optional = Optional.empty(); // this is bit safer

In [ ]:
abstract class Maybe<T> 
{
    public class Some : Maybe<T>
    {
        public Some(T v) => Value = v;
        public T Value { get; }
    }

    public class None : Maybe<T>
    { }

    public static None Nothing() => new None(); // Unfortunately we cant use None() as static method
}

// Functionality
Maybe<string> GetLogs() => Maybe<string>.Nothing();

// Usage
var content = GetLogs();
if (content is Maybe<string>.Some s)
    Console.WriteLine(s.Value);
else
    Console.WriteLine("We have nothing....");

In [ ]:
using static System.Console;

// Extension Method
static void Match<T>(this Maybe<T> v,
    Action<Maybe<T>.Some> value, Action none)
{
    if (v is Maybe<T>.Some s) value(s);
    else none();
}

GetLogs().Match(
    v => WriteLine(v),
    () => WriteLine("We have nothing...."));

### More Complete Option Type in C# with Functional Patterns

In [ ]:
abstract class Maybe<T> 
{
    public sealed class Some : Maybe<T>
    {
        public Some(T v) => Value = v;
        public T Value { get; }
    }

    public sealed class None : Maybe<T>
    { }

    public static Maybe<T> Something(T value) => new Some(value);
    public static None Nothing() => new None();

    // Bind: Chain operations that return Maybe
    public Maybe<TResult> Bind<TResult>(Func<T, Maybe<TResult>> binder) =>
        this is Some some ? binder(some.Value) : Maybe<TResult>.Nothing();

    // Map: Transform the value if present
    public Maybe<TResult> Map<TResult>(Func<T, TResult> transform) =>
        this is Some some
        ? Maybe<TResult>.Something(transform(some.Value))
        : Maybe<TResult>.Nothing();
        
    // Pattern matching: Perform different actions based on the type
    public TResult Match<TResult>(Func<T, TResult> onSome, Func<TResult> onNone) => // Notice TResult
            this is Some some ? onSome(some.Value) : onNone();

}

In [ ]:
record User(int Id, string Name, string Email);

var users = new Dictionary<int, User>
{
    { 1, new User(1, "Foo", "foo@example.com") },
    { 2, new User(2, "Bar", "bar@example.com") }
};

Maybe<User> FindUserById(Dictionary<int, User> users, int id) =>
    users.TryGetValue(id, out var user)
    ? Maybe<User>.Something(user)
    : Maybe<User>.Nothing();

var email1 = FindUserById(users, 1)
    .Map(user => user.Email)
    .Match(
        onSome: email => $"Email found: {email}",
        onNone: () => "User not found"
    );

var email2 = FindUserById(users, 3)
    .Map(user => user.Email)
    .Match(
        onSome: email => $"Email found: {email}",
        onNone: () => "User not found"
    );

var userName = FindUserById(users, 2)
    .Bind(user => Maybe<string>.Something(user.Name.ToUpper()))
    .Match(
        onSome: name => $"User name: {name}",
        onNone: () => "User not found"
    );

Console.WriteLine(email1);
Console.WriteLine(email2);
Console.WriteLine(userName);

- https://www.nuget.org/packages/Optional
    - https://github.com/nlkl/Optional/blob/master/src/Optional/Option_Maybe.cs
- https://github.com/louthy/language-ext

## F#

- https://en.wikipedia.org/wiki/Option_type

In [ ]:
let someValue = Some(42)    // Some is not a class or anything; someValue is infact Option type
let noValue = None          // https://github.com/dotnet/fsharp/blob/main/src/FSharp.Core/option.fs

let printOptionValue option =
    match option with
    | Some(x) -> printfn "The value is %d" x
    | None -> printfn "No value"

printOptionValue someValue
printOptionValue noValue

In [ ]:
let printOptionValue option =
    match option with
    | Some(x) -> printfn "The value is %d" x // to make float work we will need to replace %d with %A
    | None -> printfn "No value"

let option1: Option<int> = Some(42) // float will not work
let option2 = Some(42)
let option3: Option<int> = None

printOptionValue option1
printOptionValue option2
printOptionValue option3

- Why C# doesnt has "Optional" type
    - Nullable Types
    - Different Design Philosphy; favoring patterns

## C++

- C++ 17; gcc v8, clang 5, Visual Studio 2017
    - https://en.cppreference.com/w/cpp/utility/optional

In [ ]:
#include <iostream>
#include <optional>
#include <string>
 
// optional can be used as the return type of a factory that may fail
std::optional<std::string> create(bool b)
{
    if (b)
        return "Godzilla";
    return {}; // this is cool; its equivalen to return std::nullopt
}
 
// std::nullopt can be used to create any (empty) std::optional
auto create2(bool b) // auto was reused in C++ 11 for infered types; it was always there since C days
{
    return b ? std::optional<std::string>{"Godzilla"} : std::nullopt;
}
 
int main()
{
    std::cout << "create(false) returned "
              << create(false).value_or("empty") << '\n'; // ?? of C#
 
    // optional-returning factory functions are usable as conditions of while and if
    if (auto str = create2(true))
        std::cout << "create2(true) returned " << *str << '\n';
}

## Patterns around MayBe

- __Map__: Transform the value without unwrapping the container.
- __Bind__: Transform and flatten the value inside the container.
- __Match__: Handle different cases of a container or data structure.

- __Map__
    - Applies a function to a value wrapped in a context (e.g., a container, Option, List, etc.) and returns a new context with the transformed value; *It does not "unwrap" the context*
- __Bind__
    - Applies a function to a value in a context and flattens the result. The function itself must return a value in the same type of context; *It can "unwrap" and "flatten" nested contexts*
- __Match__
    - Allows destructuring and applying logic based on the structure of a value or its type. Commonly used with sum types like Option, Either, or discriminated unions; *Provides a clean and declarative way to handle all possible cases of a type*

In [ ]:
// Map in LINQ
var list = new List<int> { 1, 2, 3, 4 };
var doubled = list.Select(x => x * 2);

// Bind in LINQ
var even = list.SelectMany(x => x % 2 == 0 ? new[] { x } : Enumerable.Empty<int>());
even

// Match; unfortunately no equivalent in LINQ

<img src=images/select-many.png>

In [ ]:
// Match like behavior
var list = new List<int> { 1, 2, 3, 4 };

var matchResult = list
    .GroupBy(x => x % 2 == 0 ? "Even" : "Odd")
    .Select(group => new { Type = group.Key, Numbers = group.ToList() })
    .ToList();

foreach (var group in matchResult)
    Console.WriteLine($"{group.Type}: {string.Join(", ", group.Numbers)}"); // string.Join is "Fold" concept in Functional Programming

# 🔣 Algebraic Data Types (ADTs)

## Product Types

A product type is a type that combines several other types
- Used to combine multiple values into a single type
- In C#, we can use the built-in Tuple classes (Value and Reference Tuples) to create product types

In [ ]:
(int, Exception) divide(int a, int b)
{
    if (b == 0) return (0, new ArgumentException());

    return (a / b, null);
}

int result;
Exception exception;

(result, _) = divide(2, 4);
(_, exception) = divide(2, 0);
var tuple = divide(6, 7); Console.WriteLine(tuple.Item1);
//result = divide(3, 5); // Go

(result, exception) = divide(1, 2);

//if (result != 0) ?
//if (null == exception)

In [ ]:
let it : int * int * int = (1, 2, 3);

## Sum Types

Sum Type (also known as Tagged Union or Discriminated Union) A sum type is a type that can be one of several possible types
- a type that can be one of several different types
- In C#, we can implement sum types using inheritance; we can use Pattern Matching to check the type
    - https://github.com/dotnet/csharplang/issues/113
    - https://github.com/dotnet/csharplang/blob/main/proposals/rejected/discriminated-unions.md
    - https://github.com/dotnet/csharplang/blob/main/proposals/TypeUnions.md

In [ ]:
abstract record Shape;

record Circle(float Radius) : Shape;
record Rectangle(float Width, float Height) : Shape;

static double Area(Shape shape) => shape switch
{
    Circle c => Math.PI * c.Radius * c.Radius,
    Rectangle r => r.Width * r.Height,
    _ => throw new ArgumentException("Unknown shape", nameof(shape))
};

In [ ]:
abstract record WeatherResponse;

record NotFound(string City) : WeatherResponse;
record Error(string ErrorMessage) : WeatherResponse;

record Success(string City, double Temperature, string Condition) : WeatherResponse;
record Sunny(string City, double Temperature) : Success(City, Temperature, "Sunny");
record Cloudy(string City, double Temperature) : Success(City, Temperature, "Cloudy");
record Foggy(string City, double Temperature) : Success(City, Temperature, "Foggy");
record Frosty(string City, double Temperature) : Success(City, Temperature, "Frosty");

WeatherResponse GetWeatherResponse(string city) => city switch
{
    "Lahore" => new Sunny("Lahore", 49.0),
    "Faisalabad" => new Success("Faisalabad", 49.5, "Very Sunny"),
    not null => new NotFound(city),
    _ => new Error("Unknown error occurred")
};

##Result Pattern

In [ ]:
using System.Numerics;

abstract record Result;
abstract record Result<T> : Result;
record Success<T>(T Value) : Result<T>;     // Generic; one time investment
record Error(string ErrorMessage) : Result;

Result Divide<T>(T a, T b) where T : INumber<T> =>
    b == T.Zero
        ? new Error("Division by zero")
        : new Success<T>(a / b);

var result = Divide(20, 2);
if (result is Success<int> success)
        result = Divide(success.Value, 2);

- https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions
- https://fsharpforfunandprofit.com/posts/type-size-and-design

# 📋 Result/Either Pattern

## F#: Option and Result Types

- Option<T> is a product of the presence of a value (Some<T>) or the absence of a value (None)
- Result<T, E> is a product of a successful result (Ok<T>) and an error (Error<E>)

In [ ]:
let divide x y = // no generics noise; no result type noise; implicitly its returning Result<Ok<T>, Error>
    if y = 0 then
        Error "Division by zero"
    else
        Ok (x / y)

let printResult result = 
    match result with
    | Ok value -> printfn "Success: %d" value
    | Error msg -> printfn "Error: %s" msg

printResult(divide 10 2)
//printResult(divide 5.0 2.0)

In [ ]:
let inline divide x y = // the inline is telling compiler to substitute the actual function code at each call site
    if y = LanguagePrimitives.GenericZero then Error "Division by zero" // due to generic math it will work for both int and float
    else Ok (x / y)

let printResult result =
    match result with
    | Ok x -> printfn "Success: %A" x //%A for wider numeric data types
    | Error msg -> printfn "Error: %s" msg

printResult(divide 10 2)
printResult(divide 5.0 2.0)
//printResult(divide 5.0 2)

In [ ]:
type Shape =
    | Rectangle of width : float * length : float
    | Circle of radius : float

let calculateArea shape =
    match shape with
    | Rectangle (width, length) -> width * length
    | Circle radius -> System.Math.PI * radius * radius

In [ ]:
type Direction =
    | North
    | East
    | South
    | West

type Movement =
    | Moving of Direction
    | NotMoving

type Optional<'a> =
    | Something of 'a
    | Nothing

type Either<'a,'b> =
    | Left of 'a
    | Right of 'b

## C# Implementation

In [ ]:
readonly struct Result<T>
{
    enum ResultState { Null, Failure, Success }
    readonly ResultState state;

    public T Value { get; }
    public Exception Exception { get; }

    public bool IsSuccess => this.state == ResultState.Success;
    public bool IsFailure => this.state == ResultState.Failure;
    public bool IsNull => this.state == ResultState.Null;

    public Result(T value)
    {
        this.Value = value;
        this.Exception = null!;
        this.state = ResultState.Success;
    }

    public Result(Exception exception)
    {
        this.Value = default!;
        this.Exception = exception;
        this.state = ResultState.Failure;
    }

    public TR Match<TR>(Func<T, TR> onSuccess, Func<Exception, TR> onFailure, Func<TR>? onNull = null) =>
        IsSuccess ? onSuccess(Value) :
        IsFailure ? onFailure(Exception) :
        onNull is not null 
            ? onNull() 
            : throw new InvalidOperationException("Result is null, but no onNull function was provided.");

    public static implicit operator Result<T>(T? value) => value is not null ? new Result<T>(value) : new Result<T>();
    public static implicit operator Result<T>(Exception exception) => new(exception);
}

In [ ]:
record User(int ID, string Name);
Result<User> GetUser(int id)
{
    try
    {
        // Simulating user fetch
        if (id <= 0) throw new ArgumentException("Invalid ID");
        return new User(id, "User"); // Success case
    }
    catch (Exception ex)
    {
        return ex; // Failure case
    }
}

int id = 100;
var userResult = GetUser(id);
// Using Match to handle all possible states
string message = userResult.Match(
    onSuccess: user => $"Found user: {user.Name}",
    onFailure: ex => $"Error: {ex.Message}",
    onNull: () => "User not found"
);

- Exhaustive Pattern Matching
- Type Safety and Transformation
- Chain of Operations
- Clean Error Handling

## Rust

In [ ]:
fn divide(a: f64, b: f64) -> Result<f64, String> {
    if b == 0.0
        Err("Division by zero".to_string())
    else
        Ok(a / b)
}

fn main() {
    match divide(10.0, 2.0) {
        Ok(result) => println!("Result: {}", result),
        Err(e) => println!("Error: {}", e),
    }

    match divide(10.0, 0.0) {
        Ok(result) => println!("Result: {}", result),
        Err(e) => println!("Error: {}", e),
    }
}